In [ ]:
#write your names and students ids here

In [1]:
import pandas as pd
import numpy as np
import nltk
# nltk.download('wordnet')
# nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn import preprocessing

from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

import spacy
# from gensim.models import Word2Vec, KeyedVectors

from scipy.sparse import dok_matrix
# Install spaCy (run in terminal/prompt)
import sys
#!{sys.executable} -m pip install spacy
# Download spaCy's  'en' Model
#!{sys.executable} -m spacy download en

In [2]:
data=pd.read_json ('News_Category_Dataset_v2.json', lines=True)
data = data.drop(columns=['authors', 'link', 'date'])
data=data[:3000]

### Typical data analysis of a dataframe

In [3]:
data.shape

(3000, 3)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   category           3000 non-null   object
 1   headline           3000 non-null   object
 2   short_description  3000 non-null   object
dtypes: object(3)
memory usage: 70.4+ KB


In [5]:
#data.describe
data.head()

,category,headline,short_description
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ..."


In [6]:
print(data.groupby('category').size().size)
print(data.groupby('category').size())

19
category
BLACK VOICES      154
BUSINESS           21
COMEDY            187
CRIME              81
EDUCATION          15
ENTERTAINMENT     677
IMPACT             32
LATINO VOICES       8
MEDIA             113
POLITICS         1088
QUEER VOICES      150
RELIGION           21
SCIENCE            15
SPORTS             47
TECH               30
TRAVEL             23
WEIRD NEWS         86
WOMEN              74
WORLD NEWS        178
dtype: int64


we can see that many categories are similar.
Let´s merge them. We will merge the more specific ones into the more general ones in case of doubt
the code in the cell below was obtained from https://www.kaggle.com/rmisra/news-category-dataset/discussion/114275

In [7]:
data['category']=data['category'].replace({"HEALTHY LIVING": "WELLNESS",
"QUEER VOICES": "GROUPS VOICES",
"BUSINESS": "BUSINESS & FINANCES",
"PARENTS": "PARENTING",
"BLACK VOICES": "GROUPS VOICES",
"THE WORLDPOST": "WORLD NEWS",
"STYLE": "STYLE & BEAUTY",
"GREEN": "ENVIRONMENT",
"TASTE": "FOOD & DRINK",
"WORLDPOST": "WORLD NEWS",
"SCIENCE": "SCIENCE & TECH",
"TECH": "SCIENCE & TECH",
"MONEY": "BUSINESS & FINANCES",
"ARTS": "ARTS & CULTURE",
"COLLEGE": "EDUCATION",
"LATINO VOICES": "GROUPS VOICES",
"CULTURE & ARTS": "ARTS & CULTURE",
"FIFTY": "MISCELLANEOUS",
"GOOD NEWS": "MISCELLANEOUS"})

In [8]:
print(data.groupby('category').size().size)
print(data.groupby('category').size())

16
category
BUSINESS & FINANCES      21
COMEDY                  187
CRIME                    81
EDUCATION                15
ENTERTAINMENT           677
GROUPS VOICES           312
IMPACT                   32
MEDIA                   113
POLITICS               1088
RELIGION                 21
SCIENCE & TECH           45
SPORTS                   47
TRAVEL                   23
WEIRD NEWS               86
WOMEN                    74
WORLD NEWS              178
dtype: int64


In [9]:
(data.groupby('category').size()).std()

292.42275333268213

As we may observe, the data is still highly imbalanced: some categories contain about 1000 samples while others contain more than 10000. Therefore further techniques will have to be applied to
make up for this
(Using K-fold Cross-Validation?)

# NEW VERSION (with SMOTE and CountVectorizer() )

In [10]:
from sklearn.model_selection import train_test_split
# y = data['category']
y = data['category'].copy()
# FIX THIS!! X IS ONLY HEADLINES NOW
X = data.drop(columns=['category', 'short_description'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print("Number X_train dataset: ", X_train.shape)
print("Number y_train dataset: ", y_train.shape)
print("Number X_test dataset: ", X_test.shape)
print("Number y_test dataset: ", y_test.shape)
print(y_test)

Number X_train dataset:  (2100, 1)
Number y_train dataset:  (2100,)
Number X_test dataset:  (900, 1)
Number y_test dataset:  (900,)
311     ENTERTAINMENT
1025         POLITICS
1587    ENTERTAINMENT
2941         POLITICS
2980         POLITICS
            ...      
1341    ENTERTAINMENT
1612            MEDIA
757          POLITICS
39             COMEDY
1043         POLITICS
Name: category, Length: 900, dtype: object


In [11]:
# !conda install -c conda-forge imbalanced-learn
# !pip install imblearn
# !pip install -U imbalanced-learn
# !pip install delayed
from imblearn.over_sampling import SMOTE

convert text data to numeric before applying SMOTE

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(X_train.values.ravel())
X_train=vectorizer.transform(X_train.values.ravel())
X_test=vectorizer.transform(X_test.values.ravel())
X_train=pd.DataFrame(X_train.toarray())
X_test=pd.DataFrame(X_test.toarray())
print(X_train)
print(y_train)

      0     1     2     3     4     5     6     7     8     9     ...  5992  \
0        0     0     0     0     0     0     0     0     0     0  ...     0   
1        0     0     0     0     0     0     0     0     0     0  ...     0   
2        0     0     0     0     0     0     0     0     0     0  ...     0   
3        0     0     0     0     0     0     0     0     0     0  ...     0   
4        0     0     0     0     0     0     0     0     0     0  ...     0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
2095     0     0     0     0     0     0     0     0     0     0  ...     0   
2096     0     0     0     0     0     0     0     0     0     0  ...     0   
2097     0     0     0     0     0     0     0     0     0     0  ...     0   
2098     0     0     0     0     0     0     0     0     0     0  ...     0   
2099     0     0     0     0     0     0     0     0     0     0  ...     0   

      5993  5994  5995  5996  5997  5998  5999  600

In [13]:
# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
# !conda install -c conda-forge imbalanced-learn
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state = 4).fit_resample(X_train, y_train)
print(X_resampled.shape)
print(y_resampled.shape)
print('Original categories sizes %s' % data.groupby('category').size())
print('Resampled categories sizes %s' % pd.Series(index=y_resampled.values).groupby(level=0).size())

(12016, 6002)
(12016,)
Original categories sizes category
BUSINESS & FINANCES      21
COMEDY                  187
CRIME                    81
EDUCATION                15
ENTERTAINMENT           677
GROUPS VOICES           312
IMPACT                   32
MEDIA                   113
POLITICS               1088
RELIGION                 21
SCIENCE & TECH           45
SPORTS                   47
TRAVEL                   23
WEIRD NEWS               86
WOMEN                    74
WORLD NEWS              178
dtype: int64
Resampled categories sizes BUSINESS & FINANCES    751
COMEDY                 751
CRIME                  751
EDUCATION              751
ENTERTAINMENT          751
GROUPS VOICES          751
IMPACT                 751
MEDIA                  751
POLITICS               751
RELIGION               751
SCIENCE & TECH         751
SPORTS                 751
TRAVEL                 751
WEIRD NEWS             751
WOMEN                  751
WORLD NEWS             751
dtype: int64


<ipython-input-13-60803145022d>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  print('Resampled categories sizes %s' % pd.Series(index=y_resampled.values).groupby(level=0).size())


### Multinomial Naive Bayes Classifier

In [14]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_resampled, y_resampled)

MultinomialNB()

### MODEL EVALUATION

We need to create many more variations with many different features

In [15]:
from sklearn.metrics import classification_report
predicted_nb = mnb.predict(X_test)
print(X_test.shape)
print(y_test.shape)
print(classification_report(y_test, predicted_nb));

(900, 6002)
(900,)
                     precision    recall  f1-score   support

BUSINESS & FINANCES       0.00      0.00      0.00         2
             COMEDY       0.58      0.85      0.69        53
              CRIME       0.45      0.38      0.42        26
          EDUCATION       0.33      0.33      0.33         6
      ENTERTAINMENT       0.64      0.73      0.68       199
      GROUPS VOICES       0.53      0.51      0.52        88
             IMPACT       0.00      0.00      0.00        14
              MEDIA       0.41      0.31      0.35        36
           POLITICS       0.77      0.78      0.78       337
           RELIGION       0.20      0.20      0.20         5
     SCIENCE & TECH       0.00      0.00      0.00        10
             SPORTS       0.29      0.20      0.24        10
             TRAVEL       0.14      0.17      0.15         6
         WEIRD NEWS       0.18      0.06      0.09        33
              WOMEN       0.36      0.39      0.37        23
    

# BELOW IS THE PREVIOUS VERSION

## Data cleaning:

We must  apply either lemmatization or stemming to the words.
We will try to go for lemmatization first as it depends on the context of the word.

We used the following link to guide ourselves https://www.machinelearningplus.com/nlp/lemmatization-examples-python/


//TODO: preproces words to remove apostrophes for example. isn´t -> is not

In [ ]:
import spacy
from nltk.corpus import stopwords
spacy_lemmatiser = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
stop_words = stopwords.words('english')

#lemmatizer = WordNetLemmatizer()

def clean(text):

    # Define the sentence to be lemmatized
    text=text.lower() #making everything lowercase
    
    # Tokenize: Split the sentence into words
    #word_list = nltk.word_tokenize(text)
    #print(word_list)

    # Lemmatize list of words and join
    #lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    #print(lemmatized_output)
    
    # return lemmatized_output
    #return word_list
    
    # spacy Lemmatiser with POS tagging
    t_l_text = spacy_lemmatiser(text)
    lemmas = [token.lemma_ for token in t_l_text]
    
    lem_stop_text = [tok for tok in lemmas if tok not in stop_words]
            
    cleaned_text = " ".join(lem_stop_text) # Takes all the lemmas from text and joins 
    return cleaned_text

In [ ]:
data['short_description'] = data['short_description'].apply(clean)
data['headline'] = data['headline'].apply(clean)

### ENCODING CLASSES

We need to transform strings into numerical labels. Remember, if we extend our model to work with NN, 
then we should transform to one-hot encoding instead

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(data['category'])
print(list(le.classes_))
#print(le.transform(['FOOD & DRINK', 'GROUPS VOICES', 'HOME & LIVING']))

In [ ]:
data['category encoded'] = le.transform(data['category'])
data.head()

### Splitting into train and test



In [ ]:
train=data.sample(frac=0.8,random_state=200) #random state is a seed value
train.index= range(len(train))
test=data.drop(train.index)
test = test.reset_index()
print(train.shape)
print(train.index)
print(test.shape)
print(test.index)

In [ ]:
for index, new in test.iterrows():
    print(index)

In [ ]:
len(data)

### OUR MODELS

In [ ]:
#the code from assignment 2 was changed here. We had a framework to start with this way.

class FeatureCreator:
    def __init__(self):
        self.voc = {}
        self.id = {}
        self.count = 0
    
    def createFeatureSet(self, data):
        # t1 and t2 are the 2 questions we want to compare
        for index, new in data.iterrows():
            
            for w in new['headline']:
                if w in self.voc:
                    self.voc[w] += 1
                else:
                    self.voc[w] = 1
                    self.id[w] = self.count
                    self.count += 1
                    
            for w in new['short_description']:
                if w in self.voc:
                    self.voc[w] += 1
                else:
                    self.voc[w] = 1
                    self.id[w] = self.count
                    self.count += 1
                
        
class BowFeatureCreator(FeatureCreator): #runs in O(n^2)
    
     def createFeatures(self,data):
        id_len=len(self.id) #we will add this length to the index if we want to access the second question. It is made clear in the implementation

        features = dok_matrix((len(data),self.count*2)) #we will have twice the entries. One |VOC| x |2| one for the headline and the other one for the description
        print(features.shape)
        label = []
        for i, row in data.iterrows():
            for w in row['headline']:
                if w in self.id:
                    features[i,self.id[w]] += 1
                    #features[i,self.id[w] + self.count] += 1
            for w in row['short_description']:
                if w in self.id:
                    #features[i, self.id[w]]+= 1 #now there is one bow for the headline and another for the description
                    features[i,self.id[w] + self.count] += 1
            # print('i', i)
            label.append(row['category encoded'])
            
            
        #we want to make difference independent of the sentences,
        #that is, no negative values. We must take absolute value of features
        # not the most efficient thing ever here, but it does the job
        """ i=0
        for j, row in data.iterrows():
            for w in row['headline']:
                if w in self.id:
                    if features[j,self.id[w]+self.count]<0:
                        features[j,self.id[w]+self.count] *= -1
            for w in row['short_description']:
                if w in self.id:
                     if features[j,self.id[w]+self.count]<0:
                        features[j,self.id[w] + self.count] *= -1
            i+=1.
            if i%1000 == 0:
                print('iteration: ', i)"""

        #features=self.absoluteValue(features,data)
        return features,label

Calculate size of the vocabulary:

In [ ]:
%%time
features=BowFeatureCreator()
features.createFeatureSet(train)

In [ ]:
print(len(features.id))
print(len(features.voc))
print(features.count)

#print(train.shape)
print(len(train))

for k,v in features.id.items():
    if v > len(features.id):
        print(k)
        print(v)

In [ ]:
f,l = features.createFeatures(train)
print(features.count)

### Multinomial Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(f, l)

### MODEL EVALUATION

We need to create many more variations with many different features

In [ ]:
from sklearn.metrics import classification_report
testf,testl = features.createFeatures(test)
predicted_nb=mnb.predict(testf)
print(classification_report(testl, predicted_nb));

# IGNORE FROM HERE ONWARDS

//we will trin the model once we are done with the cleaning

First we split the data such that the proportion of category labels is the same in train and test. Remember, category will be our target

In [ ]:
train, test = train_test_split(data, test_size=0.2, stratify=data['category'])

Leave that for later, do not pay too much attention for now

In [ ]:

class_weights = dict(zip(np.unique(y_train), class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)))

Then you use the class weights during the training process:

train_history = model.fit( train_dataset, steps_per_epoch=n_steps, class_weight=class_weights )

In [ ]:
data.loc[1,'headline']

In [ ]:
sentences=[]
for i, row in data.iterrows():
    sentences.append(row['headline'])

model_embeddings=Word2Vec(vector_size=20, min_count=1)
model_embeddings.build_vocab(sentences, progress_per=10000)
model_embeddings.train(sentences, total_examples=model_embeddings.corpus_count, epochs=30,report_delay=1)
#model_embeddings.build(data['headlines'])


In [ ]:
model_embeddings.wv.most_similar(positive=["korea"])
model_embeddings.wv.most_similar(positive=["Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song"])